In [12]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from scipy.special import rel_entr
import numpy as np

In [13]:
validate_processed_sample = 'data/processed/validate_processed_sample.csv'
validate_processed_target = 'data/processed/validate_processed_target.csv'
validate_processed_diff = 'data/processed/validate_processed_differential_diagnosis.csv'

In [14]:
X_val = pd.read_csv(validate_processed_sample)
y_val = pd.read_csv(validate_processed_target)

dval = xgb.DMatrix(X_val, label=y_val)

del X_val
del y_val

diff_val = pd.read_csv(validate_processed_diff)
diff_val = diff_val.to_numpy()

In [21]:
bst = xgb.Booster()
model_filename = 'model/xgboost_model_7749.json'
bst.load_model(model_filename)

In [22]:
y_pred_prob = bst.predict(dval)
# predicted_classes = np.argmax(y_pred_prob, axis=1)

threshold = 0.01

DDrecall = 0
DDprecision = 0
for i in range(len(y_pred_prob)):
    overlap = 0
    diff_num = 0 
    pred_num = 0
    for pathology in range(49):
        if (y_pred_prob[i][pathology] > threshold):
            pred_num += 1
        if (diff_val[i][pathology] > threshold):
            diff_num += 1

            if (y_pred_prob[i][pathology] > threshold):
                overlap += 1

    DDrecall += overlap/diff_num
    DDprecision += overlap/pred_num

DDrecall /= len(y_pred_prob)
DDprecision /= len(y_pred_prob)
F1 = (2 * DDprecision * DDrecall) / (DDprecision + DDrecall)

In [23]:
print(DDrecall)
print(DDprecision)
print(F1)

0.018655331472074644
0.1776137301683176
0.03376429257457336
